<a href="https://colab.research.google.com/github/OmegaTrees/JupiterLab-Render/blob/main/Instagram%20uploader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install instagrapi python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.1/717.1 kB 9.8 MB/s eta 0:00:00


In [ ]:

"""
Instagram Upload Bot for Telegram - Google Colab Compatible

Setup Instructions for Google Colab:
1. Run this cell first:
   !pip install instagrapi python-telegram-bot nest_asyncio

2. Replace YOUR_TELEGRAM_BOT_TOKEN with your actual bot token
3. Run the script

The bot will handle the event loop automatically for Colab compatibility.
"""

import os
import asyncio
import logging
from pathlib import Path
from instagrapi import Client
from telegram import Update, InlineKeyboardMarkup, InlineKeyboardButton
from telegram.ext import (ApplicationBuilder, CommandHandler, MessageHandler,
                          filters, ContextTypes, ConversationHandler, CallbackQueryHandler)

logging.basicConfig(level=logging.INFO)

BOT_TOKEN = "7581268493:AAEffwjhMZHtFgWaXJau2NTq6RvJBTxusyA"
SESSION_FILE = "session.json"
MEDIA_PATH = "media"
Path(MEDIA_PATH).mkdir(exist_ok=True)

USERNAME, PASSWORD, CAPTION = range(3)

user_session = {}
media_message_tasks = {}
auto_upload_mode = {}
custom_captions = {}

async def safe_delete_file(file_path):
    """Safely delete a file with retry logic for Windows file locks"""
    max_retries = 10
    retry_delay = 0.5

    for attempt in range(max_retries):
        try:
            if os.path.exists(file_path):
                os.remove(file_path)
                logging.info(f"Successfully deleted: {file_path}")
                return True
        except PermissionError as e:
            if attempt < max_retries - 1:
                logging.warning(f"File locked, retrying in {retry_delay}s (attempt {attempt + 1}/{max_retries}): {file_path}")
                await asyncio.sleep(retry_delay)
                retry_delay *= 1.5  # Exponential backoff
            else:
                logging.error(f"Failed to delete after {max_retries} attempts: {file_path}")
                return False
        except Exception as e:
            logging.error(f"Unexpected error deleting {file_path}: {e}")
            return False
    return False

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if os.path.exists(SESSION_FILE):
        auto_status = "ON" if auto_upload_mode.get(update.effective_chat.id, False) else "OFF"
        caption = custom_captions.get(update.effective_chat.id, "No caption set")
        await update.message.reply_text(
            f"✅ Session already exists. Send media to upload!\n\n"
            f"🤖 Auto-upload: {auto_status}\n"
            f"📝 Caption: {caption}\n\n"
            f"Commands:\n"
            f"/auto - Toggle auto-upload mode\n"
            f"/caption - Set custom caption\n"
            f"/clean - Clean media files\n"
            f"/logout - Delete session"
        )
    else:
        await update.message.reply_text("👋 Welcome! Please use /login to authenticate your Instagram account.")

async def login(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Please send your credentials in the format: username:password")
    return USERNAME

async def get_credentials(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("🔄 Please wait while we log you in...")
    creds = update.message.text.split(":")
    if len(creds) != 2:
        await update.message.reply_text("❌ Invalid format. Use username:password")
        return ConversationHandler.END
    username, password = creds

    cl = Client()
    try:
        await asyncio.wait_for(asyncio.to_thread(cl.login, username, password), timeout=30)
        cl.dump_settings(SESSION_FILE)
        user_session[update.effective_chat.id] = cl
        await update.message.reply_text(f"✅ Logged in as: {username}\n\nNow use /caption to set your custom caption!")
    except asyncio.TimeoutError:
        await update.message.reply_text("⏱️ Login timeout. Please try again.")
    except Exception as e:
        await update.message.reply_text(f"❌ Login failed: {e}")
    return ConversationHandler.END

async def logout(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Delete the session file to logout"""
    if os.path.exists(SESSION_FILE):
        os.remove(SESSION_FILE)
        # Clear user data
        chat_id = update.effective_chat.id
        if chat_id in user_session:
            del user_session[chat_id]
        if chat_id in auto_upload_mode:
            del auto_upload_mode[chat_id]
        if chat_id in custom_captions:
            del custom_captions[chat_id]

        await update.message.reply_text("🚪 Successfully logged out! Session deleted.\nUse /login to login again.")
    else:
        await update.message.reply_text("❌ No active session found.")

async def toggle_auto_upload(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Toggle auto-upload mode"""
    chat_id = update.effective_chat.id
    current_mode = auto_upload_mode.get(chat_id, False)
    auto_upload_mode[chat_id] = not current_mode

    mode_text = "ON" if auto_upload_mode[chat_id] else "OFF"
    status_emoji = "✅" if auto_upload_mode[chat_id] else "❌"

    await update.message.reply_text(
        f"{status_emoji} Auto-upload mode: {mode_text}\n\n"
        f"{'Media will be uploaded automatically without asking!' if auto_upload_mode[chat_id] else 'You will be asked before uploading media.'}"
    )

async def set_caption_start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Start setting custom caption"""
    current_caption = custom_captions.get(update.effective_chat.id, "Not set")
    await update.message.reply_text(
        f"Current caption: {current_caption}\n\n"
        f"Send me your new custom caption:"
    )
    return CAPTION

async def set_caption(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Set the custom caption"""
    caption = update.message.text.strip()
    custom_captions[update.effective_chat.id] = caption
    await update.message.reply_text(f"✅ Caption set to: {caption}")
    return ConversationHandler.END

async def clean_media(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Clean up all media files in the media directory"""
    deleted_count = 0
    cancelled_tasks = 0

    # Cancel all pending tasks
    for media_path, task_data in list(media_message_tasks.items()):
        task_data['task'].cancel()
        cancelled_tasks += 1
    media_message_tasks.clear()

    # Delete all files in media directory
    if os.path.exists(MEDIA_PATH):
        for filename in os.listdir(MEDIA_PATH):
            file_path = os.path.join(MEDIA_PATH, filename)
            if os.path.isfile(file_path):
                success = await safe_delete_file(file_path)
                if success:
                    deleted_count += 1

    await update.message.reply_text(
        f"🧹 Cleanup complete!\n"
        f"• Files deleted: {deleted_count}\n"
        f"• Tasks cancelled: {cancelled_tasks}"
    )

async def upload_to_instagram(file_path, caption, cl):
    """Upload file to Instagram with proper error handling"""
    try:
        if file_path.endswith(".mp4"):
            result = await asyncio.to_thread(cl.clip_upload, file_path, caption)
        else:
            result = await asyncio.to_thread(cl.photo_upload, file_path, caption)
        return True, "Success"
    except Exception as e:
        return False, str(e)

async def handle_media(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if not os.path.exists(SESSION_FILE):
        await update.message.reply_text("⚠️ Please log in using /login first.")
        return

    cl = Client()
    cl.load_settings(SESSION_FILE)
    user_session[update.effective_chat.id] = cl

    file = update.message.video or update.message.photo[-1]
    is_video = bool(update.message.video)
    file_size = file.file_size

    if is_video and file_size > 50 * 1024 * 1024:
        await update.message.reply_text("⚠️ Video is larger than 50MB. Not allowed.")
        return

    # Use a simpler filename for Google Colab compatibility
    file_extension = ".mp4" if is_video else ".jpg"
    media_path = os.path.join(MEDIA_PATH, f"{file.file_unique_id}{file_extension}")

    file_obj = await file.get_file()
    await file_obj.download_to_drive(media_path)

    chat_id = update.effective_chat.id
    caption = custom_captions.get(chat_id, "Uploaded via Telegram Bot")

    # Check if auto-upload is enabled
    if auto_upload_mode.get(chat_id, False):
        # Auto-upload mode
        status_message = await update.message.reply_text("🤖 Auto-upload mode: Uploading to Instagram...")

        success, error = await upload_to_instagram(media_path, caption, cl)

        if success:
            await status_message.edit_text("✅ Auto-uploaded to Instagram! File will be deleted in 1 minute.")

            # Delete file after 1 minute
            async def delete_after_upload():
                await asyncio.sleep(60)
                await safe_delete_file(media_path)
                try:
                    await status_message.edit_text("✅ Auto-uploaded to Instagram! File deleted.")
                except Exception:
                    pass

            asyncio.create_task(delete_after_upload())
        else:
            await status_message.edit_text(f"❌ Auto-upload failed: {error}")
            await safe_delete_file(media_path)

    else:
        # Manual mode - ask for confirmation
        reply_message = await update.message.reply_text(
            f"📱 Media downloaded\n📝 Caption: {caption}\n\nUpload to Instagram?",
            reply_markup=InlineKeyboardMarkup([
                [
                    InlineKeyboardButton("✅ Yes", callback_data=f"yes|{media_path}"),
                    InlineKeyboardButton("❌ No", callback_data=f"no|{media_path}")
                ]
            ])
        )

        # Set timeout to auto delete after 5 minutes
        async def delete_after_timeout():
            await asyncio.sleep(300)  # 5 minutes
            await safe_delete_file(media_path)
            try:
                await reply_message.edit_text("⏳ Time expired. Media deleted.")
            except Exception:
                pass
            if media_path in media_message_tasks:
                del media_message_tasks[media_path]

        # Store both the task and the message reference
        media_message_tasks[media_path] = {
            'task': asyncio.create_task(delete_after_timeout()),
            'message': reply_message
        }

async def button_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    action, path = query.data.split("|", 1)
    cl = user_session.get(update.effective_chat.id)
    caption = custom_captions.get(update.effective_chat.id, "Uploaded via Telegram Bot")

    # Cancel deletion timer if active
    if path in media_message_tasks:
        media_message_tasks[path]['task'].cancel()
        del media_message_tasks[path]

    if action == "yes":
        await query.edit_message_text("🔄 Uploading to Instagram...")

        success, error = await upload_to_instagram(path, caption, cl)

        if success:
            await query.edit_message_text("✅ Successfully uploaded to Instagram! File will be deleted in 1 minute.")

            # Delete file after 1 minute
            async def delete_after_upload():
                await asyncio.sleep(60)
                await safe_delete_file(path)
                try:
                    await query.edit_message_text("✅ Successfully uploaded to Instagram! File deleted.")
                except Exception:
                    pass

            asyncio.create_task(delete_after_upload())
        else:
            await query.edit_message_text(f"❌ Upload failed: {error}")
            await safe_delete_file(path)
    else:
        await query.edit_message_text("❌ Upload canceled. Media deleted.")
        await safe_delete_file(path)

if __name__ == '__main__':
    app = ApplicationBuilder().token(BOT_TOKEN).build()

    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("clean", clean_media))
    app.add_handler(CommandHandler("logout", logout))
    app.add_handler(CommandHandler("auto", toggle_auto_upload))

    login_conv = ConversationHandler(
        entry_points=[CommandHandler("login", login)],
        states={USERNAME: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_credentials)]},
        fallbacks=[]
    )

    caption_conv = ConversationHandler(
        entry_points=[CommandHandler("caption", set_caption_start)],
        states={CAPTION: [MessageHandler(filters.TEXT & ~filters.COMMAND, set_caption)]},
        fallbacks=[]
    )

    app.add_handler(login_conv)
    app.add_handler(caption_conv)
    app.add_handler(MessageHandler(filters.PHOTO | filters.VIDEO, handle_media))
    app.add_handler(CallbackQueryHandler(button_handler))

    # Google Colab compatibility
    try:
        import nest_asyncio
        nest_asyncio.apply()
        print("✅ Applied nest_asyncio for Google Colab compatibility")
    except ImportError:
        print("⚠️ nest_asyncio not found. Install it for Google Colab: !pip install nest_asyncio")

    try:
        app.run_polling()
    except RuntimeError as e:
        if "event loop is already running" in str(e):
            print("🔄 Detected running event loop, starting bot in async mode...")
            import asyncio
            loop = asyncio.get_event_loop()
            if loop.is_running():
                # For Google Colab - run in background
                task = loop.create_task(app.start())
                task = loop.create_task(app.updater.start_polling())
                print("✅ Bot started successfully in Google Colab!")
                print("🤖 Bot is now running. Send /start to begin!")
            else:
                app.run_polling()
        else:
            raise e

✅ Applied nest_asyncio for Google Colab compatibility


  if event.key is 'enter':



Analyzing CLIP file "media/AgADUBMAAnFCMVU.mp4"
Generating thumbnail "media/AgADUBMAAnFCMVU.mp4.jpg"...
Analyzing CLIP file "media/AgADVRMAAnFCMVU.mp4"
Generating thumbnail "media/AgADVRMAAnFCMVU.mp4.jpg"...
Analyzing CLIP file "media/AgADZRMAAmx3YFU.mp4"
Generating thumbnail "media/AgADZRMAAmx3YFU.mp4.jpg"...
Analyzing CLIP file "media/AgADTRAAAqM0OVU.mp4"
Generating thumbnail "media/AgADTRAAAqM0OVU.mp4.jpg"...


ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 103, in handle_async_request
    return await self._connection.handle_async_req